Authored by: Aryan Mistry, Harish Kashyap

# Building a Bigram Language Model with New York Times Data

In this extended lab you will build and explore a bigram language model using a small subset of **New York Times** headlines and teasers. Bigram models estimate the probability of a word given the previous word by counting how often word pairs occur in a corpus. While far less powerful than modern neural language models, n‑gram models introduce foundational concepts such as tokenization, smoothing, and text generation.

We begin by examining our dataset and then walking through the steps of building and sampling from a bigram model. [7]

## 1 – Introduction to N‑gram Models

An **n‑gram model** approximates the probability of a word by looking at the preceding *n − 1* words. A **unigram** model considers only individual word frequencies, a **bigram** model conditions on the previous word, and a **trigram** model conditions on the previous two words. The probability of a sentence \(w_1, w_2, \dots, w_T\) under a bigram model is given by

$$P(w_1,\dots,w_T) = \prod_{t=1}^{T} P(w_t\mid w_{t-1})$$

where $P(w_t\mid w_{t-1})$ is estimated from counts in the training data. [7]

## 2 – Exploring the Dataset

The following CSV snippet contains five sample rows from a New York Times headlines dataset. Each row has a `headline`, `teaser`, and other metadata. In a real project you would load thousands of articles; here we use a small subset for demonstration.

In [ ]:
nyt_sample = """date,time,headline,URL,teaser,section
10/8/2017,6:45 AM,"North Korean Leader Hails Nuclear Arsenal as 'Powerful Deterrent'",https://www.nytimes.com/2017/10/08/world/asia/kim-jong-un-nuclear-weapons.html?partner=rss&emc=rss,"Kim Jong-un struck a defiant stance even after President Trump said diplomacy had failed to end the crisis. And South Korean officials were worried that the North would conduct a major missile or nuclear test.","NYT > Politics"
10/8/2017,8:12 AM,"Erik Prince, Blackwater Founder, Bets He Can Turn a Big Profit by Rebuilding War-Battered Wyoming",https://www.nytimes.com/2017/10/08/us/politics/erik-prince-blackwater-wyoming-senate.html?partner=rss&emc=rss,"Mr. Prince is drawing national headlines as he tries to test his credentials as a high-profile contender in a fledgling drive to oust establishment lawmakers with insurgents in the mold of President Trump.","NYT > Politics"
10/8/2017,10:00 AM,"At Queens Museum, the Director Is a Political Activist",https://www.nytimes.com/2017/10/08/arts/design/queens-museum-laura-raicovich-daca.html?partner=rss&emc=rss,"Laura Raicovich is particularly outspoken on immigration, an issue that hits close to home: Five percent of the museum's staff members are DACA recipients.","NYT > Politics"
10/8/2017,12:07 PM,"Trump Goes After Senator Bob Corker, Who Says White House Has Become an 'Adult Day Care Center'",https://www.nytimes.com/2017/10/08/us/politics/trump-corker.html?partner=rss&emc=rss,"Senator Corker, who decided not to run for re-election, said that the White House had become an 'adult day care center,' and said, 'Someone obviously missed their shift this morning.'","NYT > Politics"
10/8/2017,2:38 PM,"Pence Leaves Colts Game After Visiting Players Kneel During Anthem",https://www.nytimes.com/2017/10/08/us/politics/pence-anthem-colts.html?partner=rss&emc=rss,"Following President Trump's lead, the vice president said he could not dignify any event that disrespects our soldiers, our Flag, or our National Anthem.","NYT > Politics"
"""

In [ ]:
import pandas as pd
from io import StringIO

# Load the small NYT sample dataset
nyt_csv = StringIO(nyt_sample)
nyt_df = pd.read_csv(nyt_csv)

# Display the first few rows
nyt_df.head()

,date,time,headline,URL,teaser,section
0,10/8/2017,6:45 AM,North Korean Leader Hails Nuclear Arsenal as '...,https://www.nytimes.com/2017/10/08/world/asia/...,Kim Jong-un struck a defiant stance even after...,NYT > Politics
1,10/8/2017,8:12 AM,"Erik Prince, Blackwater Founder, Bets He Can T...",https://www.nytimes.com/2017/10/08/us/politics...,Mr. Prince is drawing national headlines as he...,NYT > Politics
2,10/8/2017,10:00 AM,"At Queens Museum, the Director Is a Political ...",https://www.nytimes.com/2017/10/08/arts/design...,Laura Raicovich is particularly outspoken on i...,NYT > Politics
3,10/8/2017,12:07 PM,"Trump Goes After Senator Bob Corker, Who Says ...",https://www.nytimes.com/2017/10/08/us/politics...,"Senator Corker, who decided not to run for re-...",NYT > Politics
4,10/8/2017,2:38 PM,Pence Leaves Colts Game After Visiting Players...,https://www.nytimes.com/2017/10/08/us/politics...,"Following President Trump's lead, the vice pre...",NYT > Politics


### Extracting Text for Language Modelling

We will build a bigram model on the combined text of the `headline` and `teaser` columns. To do this we concatenate the fields and perform basic preprocessing: lowercasing, removing punctuation, and splitting on word boundaries.

In [ ]:
import re
from collections import defaultdict, Counter
import random

# Combine headline and teaser columns into one string
text_corpus = ''
for _, row in nyt_df.iterrows():
    text_corpus += ' ' + str(row['headline']) + ' ' + str(row['teaser'])

# Tokenize: lowercase and extract word tokens
tokens = re.findall(r'\b\w+\b', text_corpus.lower())
print(f'Number of tokens: {len(tokens)}')
print(tokens[:30])

Number of tokens: 212
['north', 'korean', 'leader', 'hails', 'nuclear', 'arsenal', 'as', 'powerful', 'deterrent', 'kim', 'jong', 'un', 'struck', 'a', 'defiant', 'stance', 'even', 'after', 'president', 'trump', 'said', 'diplomacy', 'had', 'failed', 'to', 'end', 'the', 'crisis', 'and', 'south']


### Building Bigram Counts and Probabilities

---


We now count all adjacent word pairs and convert those counts into probabilities.

In [ ]:
# Build bigram counts
bigram_counts = defaultdict(Counter)
for i in range(len(tokens)-1):
    w1, w2 = tokens[i], tokens[i+1]
    bigram_counts[w1][w2] += 1

# Convert to probabilities with simple relative frequency
bigram_probs = {}
for w1, counter in bigram_counts.items():
    total = sum(counter.values())
    bigram_probs[w1] = {w2: c / total for w2, c in counter.items()}

# Inspect sample probabilities
for w in list(bigram_probs)[:50]:
    print(w, '→', bigram_probs[w])

north → {'korean': 0.5, 'would': 0.5}
korean → {'leader': 0.5, 'officials': 0.5}
leader → {'hails': 1.0}
hails → {'nuclear': 1.0}
nuclear → {'arsenal': 0.5, 'test': 0.5}
arsenal → {'as': 1.0}
as → {'powerful': 0.3333333333333333, 'he': 0.3333333333333333, 'a': 0.3333333333333333}
powerful → {'deterrent': 1.0}
deterrent → {'kim': 1.0}
kim → {'jong': 1.0}
jong → {'un': 1.0}
un → {'struck': 1.0}
struck → {'a': 1.0}
a → {'defiant': 0.16666666666666666, 'major': 0.16666666666666666, 'big': 0.16666666666666666, 'high': 0.16666666666666666, 'fledgling': 0.16666666666666666, 'political': 0.16666666666666666}
defiant → {'stance': 1.0}
stance → {'even': 1.0}
even → {'after': 1.0}
after → {'president': 0.3333333333333333, 'senator': 0.3333333333333333, 'visiting': 0.3333333333333333}
president → {'trump': 0.75, 'said': 0.25}
trump → {'said': 0.25, 'at': 0.25, 'goes': 0.25, 's': 0.25}
said → {'diplomacy': 0.25, 'that': 0.25, 'someone': 0.25, 'he': 0.25}
diplomacy → {'had': 1.0}
had → {'failed': 0.

### Generating Text like ChatGPT
#### Next word prediction

We can generate new sequences of words by repeatedly sampling the next word from the conditional distribution given the current word. Below is a function that produces a sequence of a given length starting from a seed word.

In [ ]:
import time

def generate_bigram_sequence(start_word: str, length: int = 20) -> str:
    current = start_word.lower()
    words = [current]
    for _ in range(length-1):
        next_dict = bigram_probs.get(current, None)
        if not next_dict:
            break
        candidates, probs = zip(*next_dict.items())
        current = random.choices(candidates, probs)[0]
        words.append(current)
        print(current)
        time.sleep(1)
    return ' '.join(words)

# Example generation
print(generate_bigram_sequence('korean', length=20))

leader
hails
nuclear
test
his
credentials
as
he
can
turn
a
major
missile
or
nuclear
test
his
credentials
as
korean leader hails nuclear test his credentials as he can turn a major missile or nuclear test his credentials as


## 3 – Exercises

Now it's your turn to extend the model and analyse the data. These exercises will help you deepen your understanding of n‑gram language modelling.

1. **Vocabulary size:** Compute the number of unique tokens in the corpus. How many unique tokens are there?
2. **Top unigrams:** List the 10 most frequent words in the corpus. What do you notice about them?
3. **Top bigrams:** Identify the 10 most common bigrams (word pairs) and their counts.
4. **Smoothing:** Implement add‑one (Laplace) smoothing when computing bigram probabilities. Compare the probabilities of rare bigrams with and without smoothing.
5. **Trigram model:** Extend the code to build a trigram (3‑gram) language model. Does it generate more coherent text on this small corpus?
6. **Perplexity:** Define a function to compute the perplexity of a given sentence under your bigram model. Test it on a few sentences.
7. **Experiment with seeds:** Try generating text from different seed words and lengths. How does the seed influence the output?

Use the cell below as a starting point for these exercises. Where you see TODO comments, replace them with your own code.

In [ ]:
# TODO: Implement your solutions here
# 1. Vocabulary size
# unique_tokens = ...

# 2. Top unigrams
# from collections import Counter
# unigram_counts = Counter(tokens)
# top_unigrams = ...

# 3. Top bigrams
# bigram_counter = Counter()
# for w1, w2s in bigram_counts.items():
#     for w2, cnt in w2s.items():
#         bigram_counter[(w1, w2)] = cnt
# top_bigrams = ...

# 4. Laplace smoothing
# def smoothed_probability(w1, w2, k=1):
#     ...

# 5. Trigram model
# def train_trigram(tokens):
#     ...

# 6. Perplexity
# def perplexity(sentence):
#     ...

# 7. Generate from different seeds
# print(generate_bigram_sequence('north', length=20))
# print(generate_bigram_sequence('immigration', length=20))

In [ ]:

from collections import Counter

unigram_counts = Counter(tokens)
top_unigrams = unigram_counts.most_common(10)
top_unigrams

[('the', 7),
 ('a', 6),
 ('to', 5),
 ('president', 4),
 ('trump', 4),
 ('said', 4),
 ('that', 4),
 ('as', 3),
 ('after', 3),
 ('he', 3)]

In [ ]:
from collections import Counter

bigrams = list(zip(tokens, tokens[1:]))
bigram_counter = Counter(bigrams)
top_bigrams = bigram_counter.most_common(10)
top_bigrams

[(('president', 'trump'), 3),
 (('that', 'the'), 2),
 (('corker', 'who'), 2),
 (('white', 'house'), 2),
 (('become', 'an'), 2),
 (('an', 'adult'), 2),
 (('adult', 'day'), 2),
 (('day', 'care'), 2),
 (('care', 'center'), 2),
 (('north', 'korean'), 1)]

In [ ]:
# 4. Laplace smoothing + comparison
from collections import Counter

# Ensure we have unigram and bigram counts
unigram_counts = Counter(tokens)
bigram_counter = Counter(zip(tokens, tokens[1:]))
V = len(set(tokens))  # vocabulary size

def p_ml(w1, w2):
    """Unsmoothed MLE bigram probability."""
    denom = unigram_counts[w1]
    return (bigram_counter[(w1, w2)] / denom) if denom > 0 else 0.0

def p_laplace(w1, w2, k=1):
    """Add-k (Laplace if k=1) smoothed bigram probability."""
    denom = unigram_counts[w1]
    return (bigram_counter[(w1, w2)] + k) / (denom + k * V) if denom > 0 else 0.0

# Compare rare pairs (edit as you like)
pairs_to_check = [
    ("north", "korean"),
    ("day", "care"),
    ("major", "missile"),
    ("staff", "members"),
]
comparison = [
    (w1, w2, p_ml(w1, w2), p_laplace(w1, w2, k=1)) for (w1, w2) in pairs_to_check
]
comparison  # (w1, w2, MLE, Laplace)


[('north', 'korean', 0.5, 0.013333333333333334),
 ('day', 'care', 1.0, 0.02),
 ('major', 'missile', 1.0, 0.013422818791946308),
 ('staff', 'members', 1.0, 0.013422818791946308)]

In [ ]:
from collections import Counter
import random

bigram_context_counts = Counter(zip(tokens, tokens[1:]))
trigram_counts = Counter(zip(tokens, tokens[1:], tokens[2:]))
vocab = set(tokens)

def trigram_next_probs(w1, w2):
    denom = bigram_context_counts[(w1, w2)]
    if denom == 0:
        return {}

    probs = {}
    for w3 in vocab:
        c = trigram_counts[(w1, w2, w3)]
        if c > 0:
            probs[w3] = c / denom
    return probs

def generate_trigram(seed=("the", "president"), length=20):
    seq = [seed[0], seed[1]]
    for _ in range(max(0, length - 2)):
        w1, w2 = seq[-2], seq[-1]
        probs = trigram_next_probs(w1, w2)

        if not probs:
            break  # Stop if no continuation exists

        words, weights = zip(*probs.items())
        w3 = random.choices(words, weights=weights, k=1)[0]
        seq.append(w3)

    return " ".join(seq)
generate_trigram(seed=("north","korean"), length=20)

'north korean leader hails nuclear arsenal as powerful deterrent kim jong un struck a defiant stance even after president trump'

Foundational LLMs & Transformers
1. Vaswani, A., et al. (2017). Attention is All You Need. Advances in Neural Information Processing Systems (NIPS 2017).
2. Brown, T. B., et al. (2020). Language Models are Few-Shot Learners. NeurIPS 2020.
3. Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. NAACL-HLT 2019.
4. OpenAI (2023). GPT-4 Technical Report. arXiv:2303.08774.
5. Touvron, H., et al. (2023). LLaMA 2: Open Foundation and Fine-Tuned Chat Models. Meta AI.

Generative AI & Sampling

6. Goodfellow, I., et al. (2014). Generative Adversarial Nets. NeurIPS 2014.
7. Bishop, C. M. (2006). Pattern Recognition and Machine Learning. Springer.
8. Neal, R. M. (1993). Probabilistic Inference Using Markov Chain Monte Carlo Methods. Technical Report CRG-TR-93-1, University of Toronto.

Retrieval-Augmented Generation (RAG) & Knowledge Grounding

9. Lewis, P., et al. (2020). Retrieval-Augmented Generation for Knowledge-Intensive NLP. NeurIPS 2020.
10. deepset ai (2023). Haystack: Open-Source Framework for Search and RAG Applications. https://haystack.deepset.ai
11. LangChain (2023). LangChain Documentation and Cookbook. https://python.langchain.com

Evaluation & Safety

12. Papineni, K., et al. (2002). BLEU: A Method for Automatic Evaluation of Machine Translation. ACL 2002.
13. Lin, C.-Y. (2004). ROUGE: A Package for Automatic Evaluation of Summaries. ACL Workshop 2004.
14. OpenAI (2024). Evaluating Model Outputs: Faithfulness and Grounding. OpenAI Docs.
15. Guardrails AI (2024). Open-Source Guardrails Framework. https://github.com/shreyar/guardrails

Prompt Engineering & Instruction Tuning

16. White, J. (2023). The Prompting Guide. https://www.promptingguide.ai
17. Ouyang, L., et al. (2022). Training Language Models to Follow Instructions with Human Feedback. NeurIPS 2022.

Agents & Tool Use

18. Yao, S., et al. (2022). ReAct: Synergizing Reasoning and Acting in Language Models. arXiv:2210.03629.
19. LangChain (2024). LangChain Agents and Tools Documentation.
20. Microsoft (2023). Semantic Kernel Developer Guide. https://learn.microsoft.com/en-us/semantic-kernel/
21. Google DeepMind (2024). Gemini Technical Report. arXiv:2312.11805.

State, Memory & Orchestration

22. LangGraph (2024). Stateful Agent Orchestration Framework. https://langchain-langgraph.vercel.app
23. Park, J. S., et al. (2023). Generative Agents: Interactive Simulacra of Human Behavior. arXiv:2304.03442.

Pedagogical and Course Design References

24. fast.ai (2023). fast.ai Deep Learning Course Notebooks. https://course.fast.ai
25. Ng, A. (2023). DeepLearning.AI Short Courses on Generative AI.
26. MIT 6.S191, Stanford CS324, UC Berkeley CS294-158. (2022–2024). Course Materials and Public Notebooks for ML and LLMs.